# Protocol Controller Example

This notebook tests the capablity to extend the basic controller to control a custom protocol developed in an aries agent. Che

In [1]:
%autoawait
import time
import asyncio

IPython autoawait is `on`, and set to use `asyncio`


In [2]:
from om_aries_controller.om_aries_controller import OMAriesController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8022
WEBHOOK_BASE = ""
ADMIN_URL = "http://alice-agent:8021"

# Based on the aca-py agent you wish to control
om_controller = OMAriesController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL)

In [3]:
response = await om_controller.connections.get_connections()
results = response['results']
print("Results : ", results)
if len(results) > 0:
    connection = response['results'][0]
    print("Connection :", connection)
    if connection['state'] == 'active':       
        connection_id = connection["connection_id"]
        print("Active Connection ID : ", connection_id)
    else:
        print("Connection is still progressing to active state, retry in a few moments")
else:
    print("You must create a connection")
    

Results :  [{'updated_at': '2020-09-10 07:03:30.662999Z', 'initiator': 'external', 'created_at': '2020-09-10 07:03:15.939601Z', 'accept': 'manual', 'request_id': '2f3f9a7d-4341-44b7-9bb9-6de9a9656303', 'invitation_key': '5a6zsXxQkrnrFpZhnZ6phfZMqWmbM9GMTntwcfmUQNAM', 'their_label': 'Bob', 'state': 'active', 'connection_id': '472d471b-da3b-4f71-9b94-7bcb8397c71a', 'my_did': 'QzoMbdwg9Ppt8BBr3LT81y', 'their_did': 'DKucuEdKa4HC156LpgqLNo', 'invitation_mode': 'once', 'routing_state': 'none'}]
Connection : {'updated_at': '2020-09-10 07:03:30.662999Z', 'initiator': 'external', 'created_at': '2020-09-10 07:03:15.939601Z', 'accept': 'manual', 'request_id': '2f3f9a7d-4341-44b7-9bb9-6de9a9656303', 'invitation_key': '5a6zsXxQkrnrFpZhnZ6phfZMqWmbM9GMTntwcfmUQNAM', 'their_label': 'Bob', 'state': 'active', 'connection_id': '472d471b-da3b-4f71-9b94-7bcb8397c71a', 'my_did': 'QzoMbdwg9Ppt8BBr3LT81y', 'their_did': 'DKucuEdKa4HC156LpgqLNo', 'invitation_mode': 'once', 'routing_state': 'none'}
Active Conne

In [4]:

def attach_handler(payload):
    connection_id = payload['connection_id']
    print("Handle Attachment", payload, connection_id)

message_listener = {
    "handler": attach_handler,
    "topic": "attachment"
}
loop = asyncio.get_event_loop()
loop.create_task(om_controller.listen_webhooks())
om_controller.register_listeners([message_listener], defaults=True)

In [5]:
from aiohttp import FormData


In [6]:
data = FormData()
data.add_field('file',
               open('test_file.txt', 'rb'),
               filename='test_file.txt',
               content_type='text/plain')

In [7]:
response = await om_controller.protocol.send_attachment(connection_id, data)
print('Attachment- Alice-> Bob')
print(response)


Attachment- Alice-> Bob
{'thread_id': 'abc64ece-8677-4117-a3c5-8addc244a5ba'}


In [8]:
response = await om_controller.terminate()
print(response)


None
